In [1]:
import jieba
import pandas as pd 
from collections import Counter
from pyecharts.charts import Line,Pie,Scatter,Bar,Map,Grid
from pyecharts.charts import WordCloud
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.globals import SymbolType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Funnel
import os
import numpy as np

In [15]:
df = pd.read_excel('data/basic/2022年数据分析岗位招聘数据.xlsx')
df.head()

,Unnamed: 0,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,省份,城市
0,0,数据分析师,1万-2万,上海,不限,硕士,"['数据分析', '数据挖掘', '数据建模', 'SQL', 'Python']",好人生(上海)健康科技有限公司,民营,20-99人,上海,上海
1,1,数据分析师,1.2万-1.7万,上海-徐汇区,1-3年,本科,"['Python', 'Tableau', '描述统计', 'R', 'Excel', 'P...",西安万兴服务外包有限责任公司,,20-99人,上海,上海
2,2,数据分析专员,7千-9千,上海,1-3年,本科,['数据分析'],上海乔治白实业有限公司,上市公司,100-299人,上海,上海
3,3,数据分析师,2万-3万,上海-杨浦区,不限,本科,"['Python', 'SQL', '回归分析', '方差分析']",上海八客信息科技有限公司,民营,1000-9999人,上海,上海
4,4,数据分析师,1.4万-2万,上海-宝山区,1-3年,本科,"['SQL', 'Python', 'Pandas', '决策树', 'SVM RNN', ...",成都奋途商贸有限公司,民营,100-299人,上海,上海


In [16]:
# 对工作地点数据处理
df['城市'] = df['地点'].apply(lambda x:x.split('-')[0])
df.head()

,Unnamed: 0,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,省份,城市
0,0,数据分析师,1万-2万,上海,不限,硕士,"['数据分析', '数据挖掘', '数据建模', 'SQL', 'Python']",好人生(上海)健康科技有限公司,民营,20-99人,上海,上海
1,1,数据分析师,1.2万-1.7万,上海-徐汇区,1-3年,本科,"['Python', 'Tableau', '描述统计', 'R', 'Excel', 'P...",西安万兴服务外包有限责任公司,,20-99人,上海,上海
2,2,数据分析专员,7千-9千,上海,1-3年,本科,['数据分析'],上海乔治白实业有限公司,上市公司,100-299人,上海,上海
3,3,数据分析师,2万-3万,上海-杨浦区,不限,本科,"['Python', 'SQL', '回归分析', '方差分析']",上海八客信息科技有限公司,民营,1000-9999人,上海,上海
4,4,数据分析师,1.4万-2万,上海-宝山区,1-3年,本科,"['SQL', 'Python', 'Pandas', '决策树', 'SVM RNN', ...",成都奋途商贸有限公司,民营,100-299人,上海,上海


In [17]:
# 薪资数据处理
# 去除非范围类数据
df2 = df[df['薪资范围'] != '面议']
df2 = df2[~df2.薪资范围.str.contains('/天')]
df2 = df2[~df2.薪资范围.str.contains('以下')]
df2['薪资下限'] = df2.薪资范围.apply(lambda x:x.split('-')[0])
df2['薪资上限'] = df2.薪资范围.apply(lambda x:x.split('-')[1])

In [18]:
df2.head()

,Unnamed: 0,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,省份,城市,薪资下限,薪资上限
0,0,数据分析师,1万-2万,上海,不限,硕士,"['数据分析', '数据挖掘', '数据建模', 'SQL', 'Python']",好人生(上海)健康科技有限公司,民营,20-99人,上海,上海,1万,2万
1,1,数据分析师,1.2万-1.7万,上海-徐汇区,1-3年,本科,"['Python', 'Tableau', '描述统计', 'R', 'Excel', 'P...",西安万兴服务外包有限责任公司,,20-99人,上海,上海,1.2万,1.7万
2,2,数据分析专员,7千-9千,上海,1-3年,本科,['数据分析'],上海乔治白实业有限公司,上市公司,100-299人,上海,上海,7千,9千
3,3,数据分析师,2万-3万,上海-杨浦区,不限,本科,"['Python', 'SQL', '回归分析', '方差分析']",上海八客信息科技有限公司,民营,1000-9999人,上海,上海,2万,3万
4,4,数据分析师,1.4万-2万,上海-宝山区,1-3年,本科,"['SQL', 'Python', 'Pandas', '决策树', 'SVM RNN', ...",成都奋途商贸有限公司,民营,100-299人,上海,上海,1.4万,2万


In [5]:
# 定义函数将薪资转化为数字形式
def salary_handle(word):
    if word[-1] == '万':
        num = float(word.strip('万')) * 10000
    elif word[-1] == '千':
        num = float(word.strip('千')) * 1000
    return num
df2['薪资下限'] = df2.薪资下限.apply(lambda x:salary_handle(x))
df2['薪资上限'] = df2.薪资上限.apply(lambda x:salary_handle(x))
df2['薪资均值'] = round((df2.薪资上限 + df2.薪资下限)/2,2)
df2.head()

,Unnamed: 0,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,省份,城市,薪资下限,薪资上限,薪资均值
0,0,数据分析师,1万-2万,上海,不限,硕士,"['数据分析', '数据挖掘', '数据建模', 'SQL', 'Python']",好人生(上海)健康科技有限公司,民营,20-99人,上海,上海,10000.0,20000.0,15000.0
1,1,数据分析师,1.2万-1.7万,上海-徐汇区,1-3年,本科,"['Python', 'Tableau', '描述统计', 'R', 'Excel', 'P...",西安万兴服务外包有限责任公司,,20-99人,上海,上海,12000.0,17000.0,14500.0
2,2,数据分析专员,7千-9千,上海,1-3年,本科,['数据分析'],上海乔治白实业有限公司,上市公司,100-299人,上海,上海,7000.0,9000.0,8000.0
3,3,数据分析师,2万-3万,上海-杨浦区,不限,本科,"['Python', 'SQL', '回归分析', '方差分析']",上海八客信息科技有限公司,民营,1000-9999人,上海,上海,20000.0,30000.0,25000.0
4,4,数据分析师,1.4万-2万,上海-宝山区,1-3年,本科,"['SQL', 'Python', 'Pandas', '决策树', 'SVM RNN', ...",成都奋途商贸有限公司,民营,100-299人,上海,上海,14000.0,20000.0,17000.0


In [6]:
#寻找不规范数据
print('不规范数据量为：',df.shape[0] - df2.shape[0])

不规范数据量为： 0


In [7]:
# 去除不规范值
df2 = df2.reset_index(drop = True)
df2 = df2.drop(index = 5860)

In [8]:
df2.head()

,Unnamed: 0,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,省份,城市,薪资下限,薪资上限,薪资均值
0,0,数据分析师,1万-2万,上海,不限,硕士,"['数据分析', '数据挖掘', '数据建模', 'SQL', 'Python']",好人生(上海)健康科技有限公司,民营,20-99人,上海,上海,10000.0,20000.0,15000.0
1,1,数据分析师,1.2万-1.7万,上海-徐汇区,1-3年,本科,"['Python', 'Tableau', '描述统计', 'R', 'Excel', 'P...",西安万兴服务外包有限责任公司,,20-99人,上海,上海,12000.0,17000.0,14500.0
2,2,数据分析专员,7千-9千,上海,1-3年,本科,['数据分析'],上海乔治白实业有限公司,上市公司,100-299人,上海,上海,7000.0,9000.0,8000.0
3,3,数据分析师,2万-3万,上海-杨浦区,不限,本科,"['Python', 'SQL', '回归分析', '方差分析']",上海八客信息科技有限公司,民营,1000-9999人,上海,上海,20000.0,30000.0,25000.0
4,4,数据分析师,1.4万-2万,上海-宝山区,1-3年,本科,"['SQL', 'Python', 'Pandas', '决策树', 'SVM RNN', ...",成都奋途商贸有限公司,民营,100-299人,上海,上海,14000.0,20000.0,17000.0


## 岗位需求

In [9]:
df_city=df2['省份'].value_counts()
df_city

省份
广东    989
北京    942
上海    934
江苏    454
山东    395
四川    379
浙江    311
福建    285
湖北    263
辽宁    257
河南    231
天津    210
陕西    205
湖南    200
重庆    147
吉林     60
Name: count, dtype: int64

In [10]:
job_demand = df2.省份.value_counts().sort_values(ascending = True)

In [11]:
#绘制滑动柱状图
color_js =  """new echarts.graphic.LinearGradient(0, 0, 1, 0,
    [{offset: 0, color: '#72EDF2'}, {offset: 1, color: '#5151E5'}], false)"""



x_data = job_demand.index.tolist()
y_data = job_demand.values.tolist()
b1 = (
        Bar(init_opts=opts.InitOpts())
        .add_xaxis(x_data)
        .add_yaxis('',
                   y_data ,
                   category_gap="50%",
                   label_opts=opts.LabelOpts(
                       position='top', # position 标签的位置 可选 'top'，'left'，'right'，'bottom'，'inside'，'insideLeft'，'insideRight'
                       font_size=13,
                       color= '#333333',
                       font_weight = 'bolder',  # font_weight 文字字体的粗细  'normal'，'bold'，'bolder'，'lighter'
                       font_style = 'normal',  # font_style 文字字体的风格，可选 'normal'，'italic'，'oblique'
                       formatter = '{c}'  
                    ),
                  )
        .set_series_opts(
            itemstyle_opts={
                "normal": {
                    "color": JsCode(color_js),
                    "barBorderRadius": [100, 100, 100, 100]
                }
            }
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title='数据分析岗位招聘数量省份排名',
                                       title_textstyle_opts=opts.TextStyleOpts(),
                                       pos_top='7%',pos_left = 'center'
                                     ),
            legend_opts=opts.LegendOpts(is_show=False),
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
            yaxis_opts=opts.AxisOpts(name="",
                                     name_location='middle',
                                     name_gap=40,
                                     name_textstyle_opts=opts.TextStyleOpts(font_size=16)),
                         datazoom_opts=[opts.DataZoomOpts(range_start=1,range_end=50)]
                        )

    )
b1.render_notebook()

In [12]:
#绘制地图
m1 = (
        Map()
        .add('全国数据分析岗位数量分布', [list(z) for z in zip(df_city.index.values.tolist(), df_city.values.tolist())], 'china')
        .set_global_opts(
        title_opts=opts.TitleOpts(title='全国数据分析岗位数量分布'),

        visualmap_opts=opts.VisualMapOpts(max_=100, is_piecewise=False,range_color=["white", "#72EDF2", "#5151E5"]),
        )
    )
m1.render_notebook()

## 岗位要求

In [14]:
job_experience = df2.工作经验.value_counts().sort_values(ascending = True)

In [15]:
# 绘制圆角柱状图函数
def echarts_bar(x,y,title = '主标题',subtitle = '副标题',label = '图例'):
    bar = Bar(
            init_opts=opts.InitOpts(
            # bg_color='#080b30',  # 设置背景颜色
            theme='shine',         # 设置主题

        )
    )
    bar.add_xaxis(x)
    bar.add_yaxis(label,y,
        label_opts=opts.LabelOpts(is_show=True) # 是否显示数据
        ,category_gap="50%" # 柱子宽度设置
        ) 
    bar.reversal_axis()
    bar.set_series_opts( # 自定义图表样式
        label_opts=opts.LabelOpts(
            is_show=True,
            position='right', # position 标签的位置 可选 'top'，'left'，'right'，'bottom'，'inside'，'insideLeft'，'insideRight'
            font_size=15,
            color= '#333333',
            font_weight = 'bolder',  # font_weight 文字字体的粗细  'normal'，'bold'，'bolder'，'lighter'
            font_style = 'oblique',  # font_style 文字字体的风格，可选 'normal'，'italic'，'oblique'
            ), # 是否显示数据标签
        itemstyle_opts={  
            "normal": {
                "color": JsCode(
                   """new echarts.graphic.LinearGradient(0, 0, 1, 0,
    [{offset: 0, color: '#72EDF2'}, {offset: 1, color: '#5151E5'}], false)"""
                ),       # 调整柱子颜色渐变
                'shadowBlur': 6,   # 光影大小
                "barBorderRadius": [100, 100, 100, 100],  # 调整柱子圆角弧度
                "shadowColor": "#999999", # 调整阴影颜色
                'shadowOffsetY': 2,
                'shadowOffsetX': 2,  # 偏移量
            }
        }
    )
    bar.set_global_opts(
        # 标题设置
        title_opts=opts.TitleOpts(
            title=title, # 主标题
            subtitle=subtitle, # 副标题

            title_textstyle_opts=opts.TextStyleOpts(color='#2C3B4C', font_size=20,font_weight='bolder')
        ),
        # 图例设置
        legend_opts=opts.LegendOpts(
            is_show=True, # 是否显示图例
            pos_left='right', # 图例显示位置
            pos_top='3%',  #图例距离顶部的距离
            orient='horizontal'  # 图例水平布局
        ),
        tooltip_opts=opts.TooltipOpts(
            is_show=True,  # 是否使用提示框
            trigger='axis',  # 触发类型
            trigger_on='mousemove|click',  # 触发条件，点击或者悬停均可出发
            axis_pointer_type='cross',  # 指示器类型，鼠标移动到图表区可以查看效果
        ),
        yaxis_opts=opts.AxisOpts(
            is_show=True,
            splitline_opts=opts.SplitLineOpts(is_show=False), # 分割线
            axistick_opts=opts.AxisTickOpts(is_show=False), # 刻度不显示
            axislabel_opts=opts.LabelOpts(  # 坐标轴标签配置
                font_size=13,  # 字体大小
                font_weight='bolder' # 字重
            ),
        ),   # 关闭Y轴显示
        xaxis_opts=opts.AxisOpts(
            boundary_gap=True,    # 两边不显示间隔
            axistick_opts=opts.AxisTickOpts(is_show=True),  # 刻度不显示
            splitline_opts=opts.SplitLineOpts(is_show=False),  # 分割线不显示
            axisline_opts=opts.AxisLineOpts(is_show=True),  # 轴不显示
            axislabel_opts=opts.LabelOpts(  # 坐标轴标签配置
                font_size=13,  # 字体大小
                font_weight='bolder' # 字重
            ),
        ),
    )
    return bar

In [16]:
echarts_bar(job_experience.index.tolist(),job_experience.values.tolist(),title = '数据分析岗岗位工作经验要求排名',subtitle = ' ',
            label = '工作经验').render_notebook()

In [17]:
df2['学历要求'].unique()

array(['硕士', '本科', '大专', '学历不限', '高中', '博士', '中专/中技'], dtype=object)

In [18]:
g = df2.groupby('学历要求')
g.count()['职位名称']

学历要求
中专/中技      19
博士         28
大专       1684
学历不限      127
本科       3926
硕士        465
高中         14
Name: 职位名称, dtype: int64

In [19]:
job_name =  g.count()['职位名称']
job_name

学历要求
中专/中技      19
博士         28
大专       1684
学历不限      127
本科       3926
硕士        465
高中         14
Name: 职位名称, dtype: int64

In [20]:
directions = job_name.index.tolist()
count = job_name.values.tolist()

In [21]:
#绘制环形图
c1 = (
    Pie(init_opts=opts.InitOpts())
        .add(
        '',
        [list(z) for z in zip(directions, count)],
        radius=['40%', '75%'],
        center=['40%', '50%'],
#         rosetype="radius",
        label_opts=opts.LabelOpts(is_show=True),
                    itemstyle_opts={
            "normal": {
                    "barBorderRadius": [30, 30, 30, 30],
                    'shadowBlur': 10,
                    'shadowColor': 'rgba(0,191,255,0.5)',
                    'shadowOffsetY': 1,
                    'opacity': 0.8,
                
                    }
                       },
        )    
        .set_global_opts(
            title_opts=opts.TitleOpts(title='学历要求',
                                      pos_left='1%'
                                     ),
             legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%",pos_top="25%",orient="vertical")
                        )
                .set_series_opts(
            abel_opts=opts.LabelOpts(
                formatter="{b}：{d}%",
                color="#00c6d7",
                font_size="20",
                
            ),
        )
    )
c1.render_notebook()

In [22]:
#绘制词云图
tag_array = df['岗位标签'].apply(lambda x:eval(x)).tolist()
tag_lis = []
for tag in tag_array:
    tag_lis += tag
tag_df = pd.DataFrame(tag_lis,columns = ['职位标签'])
tag_df_cnt = tag_df['职位标签'].value_counts().reset_index()
tag_df_cnt.columns = ['职位标签','计数']
word_cnt_lis = [tag for tag in zip(tag_df_cnt['职位标签'],tag_df_cnt['计数'])]


wc = (
    WordCloud()
    .add("",
    word_cnt_lis,
    shape="diamond"
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="数据分析岗位标签词云图",pos_left = 'center'),
    
    )
    
)

wc.render_notebook()

## 公司类型

In [23]:
f = df2.groupby('公司类型')
f.count()['职位名称']

公司类型
              82
上市公司         641
事业单位          43
代表处            2
其它           225
医院             2
合资           436
国企           496
外商独资         338
学校/下级学院        2
律师事务所          2
民营          3396
港澳台公司         23
社会团体          12
股份制企业        563
Name: 职位名称, dtype: int64

In [24]:
company_name =  f.count()['职位名称']
company_name

公司类型
              82
上市公司         641
事业单位          43
代表处            2
其它           225
医院             2
合资           436
国企           496
外商独资         338
学校/下级学院        2
律师事务所          2
民营          3396
港澳台公司         23
社会团体          12
股份制企业        563
Name: 职位名称, dtype: int64

In [26]:
#绘制饼图
company_type_cnt = df2[['公司类型','公司规模']].groupby('公司类型').count().sort_values(by = '公司规模',ascending = False).reset_index()
# 删除空值
company_type_cnt = company_type_cnt.drop(index = 10)
company_type_cnt.columns = ['公司类型','数量']
company_size_cnt = df2[['公司类型','公司规模']].groupby('公司规模').count().sort_values(by = '公司类型',ascending = False).reset_index()
company_size_cnt.columns = ['公司规模','数量']

pie1 = (Pie()
       .add('', [list(z) for z in zip(company_type_cnt['公司类型'], company_type_cnt['数量'])])
        .set_global_opts(
            title_opts=opts.TitleOpts(title='公司类型',
                                      pos_left='1%'
                                     ),
             legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%",pos_top="25%",orient="vertical")
                        )
       )
       
pie1.render_notebook()

In [28]:
#绘制饼图
pie2 = (Pie()
       .add('', [list(z) for z in zip(company_size_cnt['公司规模'], company_size_cnt['数量'])])
      .set_global_opts(
            title_opts=opts.TitleOpts(title='公司规模',
                                      pos_left='1%'
                                     ),
             legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%",pos_top="25%",orient="vertical")
                        )
       )
pie2.render_notebook()

In [29]:
beijing_demand = df2[df2['城市'] == '北京']['地点'].value_counts().sort_values(ascending = False)
shanghai_demand = df2[df2['城市'] == '上海']['地点'].value_counts().sort_values(ascending = False)
guangzhou_demand = df2[df2['城市'] == '广州']['地点'].value_counts().sort_values(ascending = False)

In [73]:
#绘制条形图-北京各区
c1 = (
    Bar()
    .add_xaxis(beijing_demand.index.tolist())
    .add_yaxis("岗位数", beijing_demand.values.tolist(), category_gap="60%")
    .set_series_opts(
        itemstyle_opts={
            "normal": {
                "color": JsCode(
                    """new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                offset: 0,
                color: 'rgba(0, 244, 255, 1)'
            }, {
                offset: 1,
                color: 'rgba(0, 77, 167, 1)'
            }], false)"""
                ),
                "barBorderRadius": [30, 30, 30, 30],
                "shadowColor": "rgb(0, 160, 221)",
            }
        }
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="北京市各个地区岗位分布数量情况"))

)

c1.render_notebook()

## 北上广岗位分布

In [74]:
#绘制条形图-上海各区
c2 = (
    Bar()
    .add_xaxis(shanghai_demand.index.tolist())
    .add_yaxis("岗位数", shanghai_demand.values.tolist(), category_gap="60%")
    .set_series_opts(
        itemstyle_opts={
            "normal": {
                "color": JsCode(
                    """new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                offset: 0,
                color: 'rgba(0, 244, 255, 1)'
            }, {
                offset: 1,
                color: 'rgba(0, 77, 167, 1)'
            }], false)"""
                ),
                "barBorderRadius": [30, 30, 30, 30],
                "shadowColor": "rgb(0, 160, 221)",
            }
        }
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="上海市各个地区岗位分布数量情况"))

)

c2.render_notebook()

In [30]:
#绘制条形图-广州各区
c3 = (
    Bar()
    .add_xaxis(guangzhou_demand.index.tolist())
    .add_yaxis("岗位数", guangzhou_demand.values.tolist(), category_gap="60%")
    .set_series_opts(
        itemstyle_opts={
            "normal": {
                "color": JsCode(
                    """new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                offset: 0,
                color: 'rgba(0, 244, 255, 1)'
            }, {
                offset: 1,
                color: 'rgba(0, 77, 167, 1)'
            }], false)"""
                ),
                "barBorderRadius": [30, 30, 30, 30],
                "shadowColor": "rgb(0, 160, 221)",
            }
        }
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="广州市各个地区岗位分布数量情况"))

)

c3.render_notebook()